# Tiny A11y Model Fine-tune Notebook
This notebook fine-tunes DeepSeek-Coder on WCAG + MDN accessibility datasets.

Requirements: Free GPU runtime in Colab.

In [ ]:
# Install required packages
!pip install --upgrade pip
!pip install datasets transformers peft huggingface_hub accelerate

In [ ]:
# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()  # Paste your token when prompted

In [ ]:
# Load dataset from Hugging Face
from datasets import load_dataset
dataset = load_dataset('younglim/a11y-dataset', split='train')
print(dataset[0])

In [ ]:
# Load model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# LoRA config for lightweight fine-tuning
peft_config = LoraConfig(r=8, lora_alpha=16, target_modules=['q_proj','v_proj'])
model = get_peft_model(model, peft_config)

In [ ]:
# Tokenize dataset
def tokenize(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=512)

dataset = dataset.map(tokenize, batched=True)

In [ ]:
# Setup Trainer and TrainingArguments
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    push_to_hub=True,
    hub_model_id='younglim/tiny-a11y-model'
)

trainer = Trainer(model=model, args=training_args, train_dataset=dataset)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Push the fine-tuned model to Hugging Face Hub
trainer.push_to_hub()

🎉 After running all cells, your fine-tuned Tiny A11y Model will be available at:
[https://huggingface.co/younglim/tiny-a11y-model](https://huggingface.co/younglim/tiny-a11y-model)